<a href="https://colab.research.google.com/github/VladislavGubarev/programming-practice/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_0_8_0_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение информации об объекте, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

In [ ]:
import requests
import sys
from datetime import datetime

API_KEY = "5PcQ5EFbvPtdpt7ruu1ShauouWK4kC6VfO2xofLH"

def fetch_apod():
    """Получает астрономическую картинку дня"""
    url = f"https://api.nasa.gov/planetary/apod?api_key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        print(f"\nДата: {data['date']}")
        print(f"Название: {data['title']}")
        print(f"Описание: {data['explanation']}\n")
        print(f"Ссылка на изображение: {data['url']}\n")
    else:
        print("Ошибка при получении APOD. Проверьте ключ API или попробуйте позже.")

def fetch_mars_photos():
    """Получает фотографии с марсохода"""
    date = input("Введите дату в формате ГГГГ-ММ-ДД (например, 2024-11-01): ")
    try:
        datetime.strptime(date, "%Y-%m-%d")
        url = f"https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos"
        params = {"earth_date": date, "api_key": API_KEY}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            photos = response.json().get("photos", [])
            if photos:
                print(f"\nФотографии с марсохода на {date}:")
                for photo in photos[:5]:
                    print(f"Камера: {photo['camera']['full_name']}, Ссылка: {photo['img_src']}")
            else:
                print("Фотографии за указанную дату отсутствуют.")
        else:
            print("Ошибка при получении данных. Проверьте дату или ключ API.")
    except ValueError:
        print("Неверный формат даты. Попробуйте еще раз.")

def fetch_near_earth_objects():
    """Получает данные об объектах, сближающихся с Землей"""
    date = input("Введите дату в формате ГГГГ-ММ-ДД (например, 2024-11-01): ")
    try:
        datetime.strptime(date, "%Y-%m-%d")
        url = f"https://api.nasa.gov/neo/rest/v1/feed"
        params = {"start_date": date, "end_date": date, "api_key": API_KEY}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            neos = data.get("near_earth_objects", {}).get(date, [])
            if neos:
                print(f"\nОбъекты, сближающиеся с Землей на {date}:")
                for obj in neos[:5]:
                    print(f"Имя: {obj['name']}, Диаметр (м): {obj['estimated_diameter']['meters']['estimated_diameter_max']:.2f}, Опасен: {'Да' if obj['is_potentially_hazardous_asteroid'] else 'Нет'}")
            else:
                print("Объекты за указанную дату отсутствуют.")
        else:
            print("Ошибка при получении данных. Проверьте ключ API или дату.")
    except ValueError:
        print("Неверный формат даты. Попробуйте еще раз.")

def fetch_space_weather():
    """Получает данные о космической погоде"""
    url = f"https://api.nasa.gov/DONKI/notifications?api_key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            print("\nПоследние данные о космической погоде:")
            for event in data[:5]:
                print(f"Событие: {event['messageType']}, Дата: {event['messageIssueTime']}")
        else:
            print("Нет данных о космической погоде.")
    else:
        print("Ошибка при получении данных. Проверьте ключ API или попробуйте позже.")

def main():
    while True:
        print("\nМеню:")
        print("1. Астрономическая картинка дня (APOD)")
        print("2. Фотографии с марсохода")
        print("3. Объекты, сближающиеся с Землей (ОСЗ)")
        print("4. Данные о космической погоде")
        print("5. Выход")
        choice = input("Выберите опцию (1-5): ")

        if choice == "1":
            fetch_apod()
        elif choice == "2":
            fetch_mars_photos()
        elif choice == "3":
            fetch_near_earth_objects()
        elif choice == "4":
            fetch_space_weather()
        elif choice == "5":
            print("Спасибо за использование приложения!")
            sys.exit()
        else:
            print("Неверный выбор. Попробуйте снова.")

if __name__ == "__main__":
    main()



Меню:
1. Астрономическая картинка дня (APOD)
2. Фотографии с марсохода
3. Объекты, сближающиеся с Землей (ОСЗ)
4. Данные о космической погоде
5. Выход

Дата: 2024-11-30
Название: Winter and Summer on a Little Planet
Описание: Winter and summer appear to come on a single night to this stunning little planet. It's planet Earth of course. The digitally mapped, nadir centered panorama covers 360x180 degrees and is composed of frames recorded during January and July from the Col du Galibier in the French Alps. Stars and nebulae of the northern winter (bottom) and summer Milky Way form the complete arcs traversing the rugged, curved horizon. Cars driving along on the road during a summer night illuminate the 2,642 meter high mountain pass, but snow makes access difficult during winter months except by serious ski touring. Cycling fans will recognize the Col du Galibier as one of the most famous climbs in planet Earth's Tour de France.

Ссылка на изображение: https://apod.nasa.gov/apod/image

# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:
Извлекать произведения искусства:

- Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.
Фильтровать произведения искусства:

- Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).
Отображать подробную информацию об оформлении:

- Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.
Разбивка на страницы и взаимодействие с пользователем:

- Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

In [ ]:
import requests

BASE_URL = "https://api.artic.edu/api/v1/artworks"

def fetch_artworks(page=1):
    """Извлекает список произведений искусства с разбивкой по страницам."""
    url = f"{BASE_URL}?page={page}&limit=10"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            artworks = data["data"]
            pagination = data["pagination"]
            return artworks, pagination
        else:
            print("Ошибка при запросе к API. Проверьте соединение.")
            return [], None
    except requests.RequestException as e:
        print(f"Ошибка сети: {e}")
        return [], None

def filter_artworks_by_artist(artworks, artist_name):
    """Фильтрует произведения искусства по имени художника (без учета регистра)."""
    return [art for art in artworks if artist_name.lower() in (art["artist_title"] or "").lower()]

def display_artwork_details(artwork):
    """Отображает подробную информацию о выбранном произведении искусства."""
    print("\nДетальная информация о произведении:")
    print(f"Название: {artwork.get('title', 'Не указано')}")
    print(f"Автор: {artwork.get('artist_title', 'Не указан')}")
    print(f"Дата: {artwork.get('date_display', 'Не указана')}")
    print(f"Носитель: {artwork.get('medium_display', 'Не указан')}")
    print(f"Информация: {artwork.get('thumbnail', {}).get('alt_text', 'Нет дополнительной информации')}")

def main():
    page = 1
    while True:
        artworks, pagination = fetch_artworks(page)
        if not artworks:
            print("Нет данных для отображения.")
            break

        print(f"\nСтраница {pagination['current_page']} из {pagination['total_pages']}")
        for idx, art in enumerate(artworks, start=1):
            print(f"{idx}. {art.get('title', 'Без названия')} - {art.get('artist_title', 'Автор неизвестен')}")

        print("\nДействия:")
        print("n - Следующая страница")
        print("p - Предыдущая страница")
        print("f - Фильтрация по художнику")
        print("d - Просмотреть детали произведения")
        print("q - Выход")

        choice = input("Выберите действие: ").lower()
        if choice == "n":
            if pagination["current_page"] < pagination["total_pages"]:
                page += 1
            else:
                print("Вы на последней странице.")
        elif choice == "p":
            if pagination["current_page"] > 1:
                page -= 1
            else:
                print("Вы на первой странице.")
        elif choice == "f":
            artist_name = input("Введите имя художника для фильтрации: ").strip()
            filtered_artworks = filter_artworks_by_artist(artworks, artist_name)
            if filtered_artworks:
                for idx, art in enumerate(filtered_artworks, start=1):
                    print(f"{idx}. {art.get('title', 'Без названия')} - {art.get('artist_title', 'Автор неизвестен')}")
            else:
                print("Произведения не найдены.")
        elif choice == "d":
            art_idx = input("Введите номер произведения для просмотра деталей: ")
            if art_idx.isdigit() and 1 <= int(art_idx) <= len(artworks):
                display_artwork_details(artworks[int(art_idx) - 1])
            else:
                print("Некорректный выбор.")
        elif choice == "q":
            print("Спасибо за использование программы!")
            break
        else:
            print("Неверный ввод. Попробуйте снова.")

if __name__ == "__main__":
    main()


# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


In [ ]:
import requests

COINGECKO_API_URL = "https://api.coingecko.com/api/v3"

# Инициализация портфеля
portfolio = {}

def fetch_current_prices(crypto_ids, fiat_currency="usd"):
    """Получает текущие цены на криптовалюты."""
    ids = ",".join(crypto_ids)
    url = f"{COINGECKO_API_URL}/simple/price"
    params = {"ids": ids, "vs_currencies": fiat_currency, "include_24hr_change": "true"}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Ошибка при получении данных от CoinGecko.")
        return {}

def fetch_historical_data(crypto_id, fiat_currency="usd"):
    """Получает исторические данные за последние 7 дней."""
    url = f"{COINGECKO_API_URL}/coins/{crypto_id}/market_chart"
    params = {"vs_currency": fiat_currency, "days": 7}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()["prices"]
    else:
        print("Ошибка при получении исторических данных.")
        return []

def display_portfolio(prices):
    """Отображает портфель и его стоимость."""
    print("\nВаш портфель:")
    total_value = 0
    for crypto, amount in portfolio.items():
        price = prices.get(crypto, {}).get("usd", 0)
        change = prices.get(crypto, {}).get("usd_24h_change", 0)
        value = price * amount
        total_value += value
        print(f"- {crypto.capitalize()}: {amount} шт., Цена: ${price:.2f}, Изменение: {change:.2f}%, Стоимость: ${value:.2f}")
    print(f"Общая стоимость портфеля: ${total_value:.2f}")

def add_to_portfolio():
    """Добавляет криптовалюту в портфель."""
    crypto = input("Введите название криптовалюты (например, bitcoin): ").lower()
    amount = float(input(f"Введите количество {crypto}: "))
    portfolio[crypto] = portfolio.get(crypto, 0) + amount
    print(f"{crypto} добавлено в портфель.")

def remove_from_portfolio():
    """Удаляет криптовалюту из портфеля."""
    crypto = input("Введите название криптовалюты для удаления: ").lower()
    if crypto in portfolio:
        del portfolio[crypto]
        print(f"{crypto} удалено из портфеля.")
    else:
        print("Криптовалюта не найдена в портфеле.")

def show_historical_data():
    """Отображает исторические данные криптовалюты."""
    crypto = input("Введите название криптовалюты (например, bitcoin): ").lower()
    data = fetch_historical_data(crypto)
    if data:
        print(f"\nИсторические данные для {crypto.capitalize()}:")
        for day in data:
            timestamp, price = day
            date = requests.utils.formatdate(timestamp // 1000)
            print(f"{date}: ${price:.2f}")
    else:
        print("Данные недоступны.")

def main():
    while True:
        print("\nМеню:")
        print("1. Посмотреть текущие цены на криптовалюты")
        print("2. Добавить криптовалюту в портфель")
        print("3. Удалить криптовалюту из портфеля")
        print("4. Посмотреть портфель и его стоимость")
        print("5. Отобразить исторические данные криптовалюты")
        print("6. Выход")

        choice = input("Выберите опцию (1-6): ")
        if choice == "1":
            cryptos = list(portfolio.keys()) or ["bitcoin", "ethereum", "dogecoin"]
            prices = fetch_current_prices(cryptos)
            for crypto, info in prices.items():
                print(f"{crypto.capitalize()}: ${info['usd']:.2f}, Изменение: {info['usd_24h_change']:.2f}%")
        elif choice == "2":
            add_to_portfolio()
        elif choice == "3":
            remove_from_portfolio()
        elif choice == "4":
            cryptos = list(portfolio.keys())
            if cryptos:
                prices = fetch_current_prices(cryptos)
                display_portfolio(prices)
            else:
                print("Ваш портфель пуст.")
        elif choice == "5":
            show_historical_data()
        elif choice == "6":
            print("Спасибо за использование программы!")
            break
        else:
            print("Неверный ввод. Попробуйте снова.")

if __name__ == "__main__":
    main()


# Дополнительно: Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

In [ ]:
#  А здесь код